In [22]:
import os
import math
import pandas as pd

In [99]:

def corpusPoint(age1=30, age2=50, unit=150000,  rr=0.13, initial=0):
  n=age2-age1 

  yearly = 12*unit 

  futureCorpus = initial * math.pow(1+rr,n)
  # (yearly*math.pow(1+rr,n)) + (yearly*math.pow(1+rr,n-1)) + (yearly*math.pow(1+rr,n-2))
  for i in range(1,n):
    futureCorpus+=  (yearly * math.pow(1+rr,n-i) )

  return futureCorpus



def corpusGrowthTrend(age1=30, age2=60, unit=150000,  rr=0.13, initial=0):
    n=age2-age1 
    yearly = 12*unit 

    corpusGrowth=[]
    corpusGrowth.append([age1, 0])
    # (yearly*math.pow(1+rr,n)) + (yearly*math.pow(1+rr,n-1)) + (yearly*math.pow(1+rr,n-2))
    
    for i in range(1,n):
        corpusGrowth.append([age1+i, round(corpusGrowth[i-1][1]*(1+rr),2)  + unit*12 ])
        
    return corpusGrowth



def corpusGrowthTrendVsRR(age1=30, age2=60, unit=150000):
    n=age2-age1 
    yearly = 12*unit 

    corpusGrowth=[]
    corpusGrowth.append([age1, 0,0,0,0,0,0,0,0,0,0])
    # (yearly*math.pow(1+rr,n)) + (yearly*math.pow(1+rr,n-1)) + (yearly*math.pow(1+rr,n-2))
    
    for i in range(1,n):
        j=0.10
        k=1
        row=[age1+i]
        while (j<=0.15):
            row.append(round(corpusGrowth[i-1][k]*(1+j),2)  + unit*12 )
            j+=0.005
            k+=1
        corpusGrowth.append(row)
    return corpusGrowth

In [114]:

# corpusGrowthTrend(age1=30, age2=55, unit=130000,  rr=0.13, initial=0)

# corpusGrowthTrendVsRR(age1=30, age2=55, unit=130000)
pd.options.display.float_format = '{:20,.0f}'.format
df=pd.DataFrame(corpusGrowthTrendVsRR(age1=30, age2=55, unit=130000))

df

,0,1,2,3,4,5,6,7,8,9,10
0,30,0,0,0,0,0,0,0,0,0,0
1,31,"1,560,000","1,560,000","1,560,000","1,560,000","1,560,000","1,560,000","1,560,000","1,560,000","1,560,000","1,560,000"
2,32,"3,276,000","3,283,800","3,291,600","3,299,400","3,307,200","3,315,000","3,322,800","3,330,600","3,338,400","3,346,200"
3,33,"5,163,600","5,188,599","5,213,676","5,238,831","5,264,064","5,289,375","5,314,764","5,340,231","5,365,776","5,391,399"
4,34,"7,239,960","7,293,402","7,347,180","7,401,297","7,455,752","7,510,547","7,565,683","7,621,162","7,676,985","7,733,152"
5,35,"9,523,956","9,619,209","9,715,370","9,812,446","9,910,442","10,009,365","10,109,222","10,210,019","10,311,762","10,414,459"
6,36,"12,036,352","12,189,226","12,344,061","12,500,877","12,659,695","12,820,536","12,983,421","13,148,372","13,315,409","13,484,555"
7,37,"14,799,987","15,029,095","15,261,908","15,498,478","15,738,858","15,983,103","16,231,266","16,483,402","16,739,567","16,999,816"
8,38,"17,839,985","18,167,150","18,500,717","18,840,803","19,187,521","19,540,991","19,901,330","20,268,661","20,643,106","21,024,789"
9,39,"21,183,984","21,634,700","22,095,796","22,567,495","23,050,024","23,543,615","24,048,503","24,564,930","25,093,141","25,633,384"
